In [1]:
import os
import sys
import matplotlib.pyplot as plt
import cv2
import numpy as np
import openpyxl

FEATURE 2

In [6]:
def find_longest_orthogonal_lines(mask):
    # Find contours in the segmentation mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Select the largest contour, which should correspond to the insect
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
    
        # Calculate the longest orthogonal lines
        rect = cv2.minAreaRect(largest_contour)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        
        # Calculate the lengths of the sides of the bounding box
        side_lengths = [np.linalg.norm(box[i] - box[(i+1) % 4]) for i in range(4)]
        side_lengths = sorted(side_lengths)
        
        # The ratio between the shortest and the longest of the two orthogonal lines
        ratio = side_lengths[0] / side_lengths[-1]
        
        return ratio, box
    else:
        return None, None

def process_images(image_folder, mask_folder, output_file):
    # Print current working directory for debugging
    print(f"Current working directory: {os.getcwd()}")
    print(f"Image folder path: {image_folder}")
    print(f"Mask folder path: {mask_folder}")

    if not os.path.exists(image_folder):
        print(f"Error: Image folder {image_folder} does not exist.")
        return
    if not os.path.exists(mask_folder):
        print(f"Error: Mask folder {mask_folder} does not exist.")
        return

    image_files = os.listdir(image_folder)
    results = []

    for image_file in image_files:
        if image_file.lower().endswith('.jpg'):
            mask_file = image_file.replace('.jpg', '.tif').replace('.JPG', '.tif')
            mask_path = os.path.join(mask_folder, mask_file)
            image_path = os.path.join(image_folder, image_file)
            print(f"Processing {image_file} with mask {mask_path}")
            if os.path.exists(mask_path):
                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                image = cv2.imread(image_path)
                if mask is not None:
                    ratio, box = find_longest_orthogonal_lines(mask)
                    if ratio is not None:
                        results.append((image_file, ratio, box, image))
            else:
                print(f"Mask not found: {mask_path}")

    # Save the results to a CSV file
    if results:
        np.savetxt(output_file, [(res[0], res[1]) for res in results], fmt='%s,%.6f', delimiter=',', header='Image,Ratio', comments='')
        print(f'Feature extraction completed. Results saved to {output_file}')
    else:
        print("No valid results to save.")
    
    return results

if __name__ == "__main__":
    image_folder = 'train/images_1_to_250'
    mask_folder = 'train/masks'
    output_file = 'train/ratios.csv'
    
    results = process_images(image_folder, mask_folder, output_file)

Current working directory: /Users/noejalabert/Documents/GitHub/noe-test-fork/dataetia/data_processing
Image folder path: train/images_1_to_250
Mask folder path: train/masks
Error: Image folder train/images_1_to_250 does not exist.


In [8]:
# Fonction pour visualiser les résultats
def visualize_results(image, box, ratio, image_file):
    plt.figure()
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.plot(box[:, 0], box[:, 1], 'r-', linewidth=2)
    plt.title(f"{image_file} - Ratio: {ratio:.2f}")
    plt.show()

In [9]:
# Définir les chemins
image_folder = '../train/images_1_to_250'
mask_folder = '../train/masks'
output_file = 'ratios_orthogonal.csv'
excel_file = '../train/classif.xlsx'

# Traiter les images et obtenir les résultats
results = process_images(image_folder, mask_folder, output_file)

Current working directory: /Users/noejalabert/Documents/GitHub/noe-test-fork/dataetia/data_processing
Image folder path: ../train/images_1_to_250
Mask folder path: ../train/masks
Processing 63.JPG with mask ../train/masks/63.tif
Mask not found: ../train/masks/63.tif
Processing 189.JPG with mask ../train/masks/189.tif
Mask not found: ../train/masks/189.tif
Processing 77.JPG with mask ../train/masks/77.tif
Mask not found: ../train/masks/77.tif
Processing 162.JPG with mask ../train/masks/162.tif
Mask not found: ../train/masks/162.tif
Processing 176.JPG with mask ../train/masks/176.tif
Mask not found: ../train/masks/176.tif
Processing 88.JPG with mask ../train/masks/88.tif
Mask not found: ../train/masks/88.tif
Processing 228.JPG with mask ../train/masks/228.tif
Mask not found: ../train/masks/228.tif
Processing 214.JPG with mask ../train/masks/214.tif
Mask not found: ../train/masks/214.tif
Processing 200.JPG with mask ../train/masks/200.tif
Mask not found: ../train/masks/200.tif
Processing 

In [10]:
# Prendre le résultat de la première image
if results:
    first_result = results[0]
    image_file, ratio, box, image = first_result
    
    # Visualiser le résultat pour la première image
    visualize_results(image, box, ratio, image_file)

    # Charger le fichier Excel
    workbook = openpyxl.load_workbook(excel_file)
    sheet = workbook.active

    # Ajouter les résultats dans le fichier Excel
    new_row = [image_file, ratio]
    sheet.append(new_row)

    # Sauvegarder les modifications dans le fichier Excel
    workbook.save(excel_file)
    print(f"Results for {image_file} saved to {excel_file}")
else:
    print("No valid results to save.")

No valid results to save.
